 0. Import the relevant libraries and packages

In [219]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np

# Creating the choice set
From   
![image.png](https://cdn.jamanetwork.com/ama/content_public/journal/psych/938422/m_yoi190087ea.png?Expires=1691689740&Signature=OvJ06Ayrw5fJLEycmRUj9I6GyITQwp89ZN5rVQDOm8dU1z6ZvhwNCwFV8yKnBDDEgxFqnaUQWgV7viAKHn5NpRdiVPo-Rgc8kjwYTn-0dnbudBGSoeVtaw8gTSIUiLLnWWVj2sPafVJA9NhK3bivpgCt2RLxI2UJRYUuNUmrrx1CFwyJTKYPlljeVSSEZ3Iwc9dNmvZ4p71MmTQSt8a9KcXSmZ0r0IZhQ187TDnqEFIiAN3mv9A7snMUAnhROj0Fg1p5COdoTVA9Z3RVNw8AT8QgJNSKl44244OjAauQwcnETbDdh2NPeCv4NGXYVclFTopaMriRPqUO4y-QPdDQjw__&Key-Pair-Id=APKAIE5G5CRDK6RD3PGA)

We now have all the elements to calculate subjective value.


Code by: Santiago Guardo

Supervision: Ricardo Pizarro and Silvia Lopez

In [220]:
''' Input should have this form

cols = ['crdm_lott_amt', 'crdm_lott_p', 'crdm_amb_lev', 'alpha', 'beta', 'gamma']

df_indif = pd.DataFrame([[20, 50, 0, 0.495323857, 0.495323857, 4.100226383]], columns= cols)
print(df_indif)
filename = 'csv/example_input.csv'
df_indif.to_csv(filename)

'''
subject_SE = 'csv/subject_input.csv'
df = pd.read_csv(subject_SE) 
df

,Unnamed: 0,crdm_lott_amt,crdm_lott_p,crdm_amb_lev,alpha,beta,gamma
0,0,20.0,50.0,0.0,0.495324,0.22168,4.100226


In [221]:
#Declare the variables
p = df['crdm_lott_p'].values[0]/100  #This is the probability of reward
v = df['crdm_lott_amt'].values[0] #The amount to be gained
alpha = df['alpha'].values[0] #Participant's or ADO alpha parameter
beta = df['beta'].values[0] #Participant's or ADO beta parameter
A= df['crdm_amb_lev'].values[0] #Ambiguity level
Vmax= 50.0 #The maximum amount we can pay to participants
#SV calculation and our probabilities options
sv = (p - beta*A/2)*v**alpha  
p_options = [0.87, 0.75, 0.62, 0.5, 0.38, 0.25, 0.13]

#Confirm the SE trial structure
print ('The probability of reward: ' + str(p))
print ('The amount to be earned: ' + str(v))
print ('The alpha parameter from the model: ' + str(alpha))
print ('The beta parameter from the model: ' + str(beta))
print ('The ambiguity level of the SE trial: ' + str(A))
print('The SV of the equivalence trial is ' + str(sv))


The probability of reward: 0.5
The amount to be earned: 20.0
The alpha parameter from the model: 0.495323857
The beta parameter from the model: 0.221680178
The ambiguity level of the SE trial: 0.0
The SV of the equivalence trial is 2.204962459791904


From the SV, we compute V, using the formula:

# $v = \left(\frac{SV}{p-\frac{\beta A}{2}}\right)^\frac{1}{\alpha}$


# 2. Create the choice set. 
For each probability level: 
- We want to make 3 trials around the point of subjective equality. 

- We want 2 extreme trials. One at $50 and one at the -SV of the 50\$ trial for each probability option. 

- We want 4 intermediate trials.  


In [239]:
#First, we compute the Subjective indiference trials.

def SE_options(sv, alpha, beta, A): #This function take as parameters SV, alpha, beta and ambiguity level
  valuesP = {} #The output will be an empty dictionary with the probabilities as keys and rewards as values.
  for p in p_options: #A loop to iterate through all the probabilities optins.
    v_new = (sv / (p-beta*A/2)) ** (1 / alpha)
    if v_new <= Vmax:
      valuesP[p] = v_new #for every p it includes the calculated value
  return valuesP


new_choicesp = SE_options(sv, alpha,beta,A)
SE_trials = {k: round(v, 2) for k, v in new_choicesp.items()}
print('Trials with the same subjective value for all of our Prob options: \n' + str(SE_trials))


Trials with the same subjective value for all of our Prob options: 
{0.87: 6.54, 0.75: 8.82, 0.62: 12.95, 0.5: 20.0, 0.38: 34.81}


- Next, we create new trials by adding and substracting \$2 from the SE trials values.

In [240]:
#Centered around subjective indifference +/- and $2
def centered ():
  centrials2 = {}
  for k,v1 in SE_trials.items():
    centrials2[k] = [float(v1) - 2.0,float(v1) + 2.0]
  return centrials2

centrials1 = centered()
print("Trials centered + - $2 around SE \n" + str(centrials1))

Trials centered + - $2 around SE 
{0.87: [4.54, 8.54], 0.75: [6.82, 10.82], 0.62: [10.95, 14.95], 0.5: [18.0, 22.0], 0.38: [32.81, 36.81]}


In [241]:
#Trials around SE (including the SE)
centrials = {}
for prob in SE_trials:
    value1 = float(SE_trials[prob]) #We convert the values in SE_trials into floats
    value2 = centrials1[prob] #We call the values from the centrials 1 dict. This have the plus/minus $2 but without the SE trials
    centrials[prob] = [value1] + value2 #Join the SE trials and the plus/minus $2 tirals
centrials
print('Trials at SE and + - \$2: \n', centrials)

Trials at SE and + - \$2: 
 {0.87: [6.54, 4.54, 8.54], 0.75: [8.82, 6.82, 10.82], 0.62: [12.95, 10.95, 14.95], 0.5: [20.0, 18.0, 22.0], 0.38: [34.81, 32.81, 36.81]}


In [242]:
CentTrials = pd.DataFrame(centrials.items(), columns=['p_reward', 'value_reward'])
CentTrials['category'] = 'Cent_SE'
CentTrials = CentTrials.explode('value_reward')

#Organizing
col = CentTrials.pop('category')
CentTrials.insert(0, col.name,col)
CentTrials


,category,p_reward,value_reward
0,Cent_SE,0.87,6.54
0,Cent_SE,0.87,4.54
0,Cent_SE,0.87,8.54
1,Cent_SE,0.75,8.82
1,Cent_SE,0.75,6.82
1,Cent_SE,0.75,10.82
2,Cent_SE,0.62,12.95
2,Cent_SE,0.62,10.95
2,Cent_SE,0.62,14.95
3,Cent_SE,0.50,20.0


**Extreme trials:**

To create trials far way from the subjective equality point (SEP). We first make trials with a Value of reward of $50, which is the maximum amount we can pay to participants. Thus this is the highest subjective value we can reach.

In [243]:
extrpos = {} #Create an empty dict
for p in CentTrials['p_reward'].unique(): #Iterate through the probability levels
    extrpos[p] = 50 #Assign them the 50 dollars value
print('Trials with the maximum Reward: \n', extrpos)

Trials with the maximum Reward: 
 {0.87: 50, 0.75: 50, 0.62: 50, 0.5: 50, 0.38: 50}


Now, let's find out what Value produces that SV_evil for all the probability levels. Those are going to be the extreme negative trials

       If SV<0:
# $v_{\text{evil}} = - \left( \frac{\left| SV \right|}{p - \frac{\beta \cdot A}{2}} \right) ^{\frac{1}{\alpha}}$

**BIG ASSUMPTION**
- In the previous formula, we are assuming that losses are represented equally than gains. 

In [249]:
#Let's make some calculations first

df1 = pd.DataFrame(SE_trials.items(), columns=['p_reward', 'value_reward'])
df1['SV_SE_reward'] = (df1['p_reward'] - beta*A/2)*df1['value_reward']**alpha #SV for each SE trial. Note how close it is to SV_sage
df1['SV_safe'] = 5.0**alpha 
df1['MaxReward'] = Vmax
df1['SV_MaxReward'] = (df1['p_reward'] - beta*A/2)*Vmax**alpha #SV of paying $50 at each probability level
df1['deltaSV'] = df1['SV_MaxReward'] - df1['SV_SE_reward'] 
df1['category'] = 'MaxPos'
#df1['SV_evil (SV_SE - SV_MaxReward)'] = df1['SV_SE_reward'] - df1['deltaSV']  #SV_evil is the SV difference between the SV

#Exclude those trials for whom the SV of the maximum reward is less than the SV of the SE
#This is the same as excluding those trials with SE points greater than $50
df1 = df1[df1['SV_MaxReward']> df1['SV_SE_reward']]
ExtrPos = df1[['category','p_reward','value_reward']]

ExtrPos

,category,p_reward,value_reward
0,MaxPos,0.87,6.54
1,MaxPos,0.75,8.82
2,MaxPos,0.62,12.95
3,MaxPos,0.50,20.00
4,MaxPos,0.38,34.81


In [290]:
#Intermediate pos and neg trials

p_options = [0.87, 0.75, 0.62, 0.5, 0.38, 0.25, 0.13]
alpha = 0.495323857
beta = 0    
num_steps = 3
# define method to get deltaSV_max for different probabilities
SV_int = {}
intpos = {}
for p in ExtrPos['p_reward'].unique(): 
    SV_max = (p - beta*A/2)*Vmax**alpha
    SV_steps = np.linspace(0,SV_max,num= num_steps + 2)
    SV_steps = SV_steps[1:-1].tolist()
    SV_int[p] = SV_steps
    v_steps = [(sv1 / (p - beta * A / 2)) ** (1 / alpha) for sv1 in SV_steps]
    intpos[p] = v_steps
intpos

print(SV_int)
print('\n',intpos)

{0.87: [1.5100788911669611, 3.0201577823339223, 4.530236673500884], 0.75: [1.301792147557725, 2.60358429511545, 3.905376442673175], 0.62: [1.076148175314386, 2.152296350628772, 3.228444525943158], 0.5: [0.8678614317051501, 1.7357228634103001, 2.60358429511545], 0.38: [0.659574688095914, 1.319149376191828, 1.978724064287742]}

 {0.87: [3.0442648575854894, 12.337473115645462, 27.972645634325314], 0.75: [3.044264857585489, 12.337473115645459, 27.972645634325303], 0.62: [3.044264857585489, 12.337473115645459, 27.972645634325303], 0.5: [3.0442648575854894, 12.337473115645462, 27.972645634325303], 0.38: [3.0442648575854894, 12.337473115645462, 27.972645634325303]}


In [298]:
IntPos = pd.DataFrame(intpos.items(), columns=['p_reward', 'value_reward'])
IntPos = IntPos.explode('value_reward')
IntPos['category'] = 'IntPos'
IntPos

,p_reward,value_reward,category
0,0.87,3.044265,IntPos
0,0.87,12.337473,IntPos
0,0.87,27.972646,IntPos
1,0.75,3.044265,IntPos
1,0.75,12.337473,IntPos
1,0.75,27.972646,IntPos
2,0.62,3.044265,IntPos
2,0.62,12.337473,IntPos
2,0.62,27.972646,IntPos
3,0.50,3.044265,IntPos


In [ ]:
for p in ExtrPos['p_reward'].unique():
        v_steps = [(sv1 / (p - beta * A / 2)) ** (1 / alpha) for sv1 in SV_steps]
    #print(p)
        print(v_steps)
    #print(SV_steps)
        intpos[p] = v_steps
    #for p in ExtrPos['p_reward'].unique():
     #   v_steps = [(sv1 / (p - beta * A / 2)) ** (1 / alpha) for sv1 in SV_steps]
      #  intpos[p] = v_steps
intpos


In [ ]:

deltasv1 = np.linspace(0,SV_max,num=5)
deltasv1 = deltasv1[1:-1].tolist()
deltasv_neg = [-1.0*d for d in deltasv1]
deltasv = sorted(deltasv1 + deltasv_neg)

SV_target = [d + (5**alpha) for d in deltasv] # Add SV_safe = 5.0**alpha to each element to get SV_lott

print(SV_target)



In [228]:
#Create a dictionary using Probability as key and SV_evil as a value
svevil = df1.set_index('p_reward')['SV_evil (SV_SE - SV_MaxReward)'].to_dict() 
svevil

{0.87: -1.6294444870178513,
 0.75: -0.7975156961353616,
 0.62: 0.10456301490286268,
 0.5: 0.9384791927632081,
 0.38: 1.7718818572597947,
 0.25: 2.6740976347261305,
 0.13: 3.507353041173875}

In [229]:
def ExtrNeg_options(svevil, alpha, beta, A):
    NegvaluesP = {}  # The output will be an empty dictionary with the probabilities as keys and rewards as values.
    for p, sv1 in svevil.items():  # A loop to iterate through all the probabilities options.
        if sv1 < 0.0:
            v_evil = - (abs(sv1) / (p - beta * A / 2)) ** (1 / alpha)  # Assuming losses are represented equally than gains!
        else:
            v_evil = (sv1 / (p - beta * A / 2)) ** (1 / alpha)
        NegvaluesP[p] = v_evil  # for every p it includes the calculated value

    return NegvaluesP

# Call the function and store the result
NegvaluesP = ExtrNeg_options(svevil, alpha, beta, A)
extrneg = {k: round(v, 2) for k, v in NegvaluesP.items()}
print('Trials with the least SV for every prob option: \n' + str(extrneg))


Trials with the least SV for every prob option: 
{0.87: -3.55, 0.75: -1.13, 0.62: 0.03, 0.5: 3.57, 0.38: 22.38, 0.25: 119.65, 0.13: 774.63}


In [230]:
# Extreme trials. Convert the dictionary to a DataFrame
#ExtrPos = pd.DataFrame(extrpos.items(), columns=['p_reward', 'value_reward'])
ExtrNeg = pd.DataFrame(extrneg.items(), columns=['p_reward', 'value_reward'])
ExtrTrials = pd.concat([ExtrPos,ExtrNeg], ignore_index=True)
ExtrTrials

,p_reward,value_reward
0,0.87,6.54
1,0.75,8.82
2,0.62,12.95
3,0.50,20.00
4,0.38,34.81
5,0.25,81.05
6,0.13,303.48
7,0.87,-3.55
8,0.75,-1.13
9,0.62,0.03


In [231]:

ExtrTrials['category'] = 'ExtrTrials'
ExtrTrials['SV_reward'] = (ExtrTrials['p_reward'] - beta*A/2)*ExtrTrials['value_reward']**alpha 

ExtrTrials['SV_safe'] = 5.0**alpha
ExtrTrials['MaxReward'] = 50
ExtrTrials['SV_MaxReward'] = (ExtrTrials['p_reward'] - beta*A/2)*50**alpha 

ExtrTrials['deltaSV'] = ExtrTrials['SV_MaxReward'] - ExtrTrials['SV_safe']
#ExtrTrials['SV_evil'] = ExtrTrials['SV_SE_reward'] - ExtrTrials['deltaSV'] 


#Organizing
col = ExtrTrials.pop('category')
ExtrTrials.insert(0, col.name,col)
ExtrTrials


,category,p_reward,value_reward,SV_reward,SV_safe,MaxReward,SV_MaxReward,deltaSV
0,ExtrTrials,0.87,6.54,2.205436,2.219303,50,6.040316,3.821013
1,ExtrTrials,0.75,8.82,2.204826,2.219303,50,5.207169,2.987866
2,ExtrTrials,0.62,12.95,2.204578,2.219303,50,4.304593,2.085290
3,ExtrTrials,0.50,20.00,2.204962,2.219303,50,3.471446,1.252143
4,ExtrTrials,0.38,34.81,2.205090,2.219303,50,2.638299,0.418996
5,ExtrTrials,0.25,81.05,2.204910,2.219303,50,1.735723,-0.483580
6,ExtrTrials,0.13,303.48,2.204964,2.219303,50,0.902576,-1.316727
7,ExtrTrials,0.87,-3.55,NaN,2.219303,50,6.040316,3.821013
8,ExtrTrials,0.75,-1.13,NaN,2.219303,50,5.207169,2.987866
9,ExtrTrials,0.62,0.03,0.109163,2.219303,50,4.304593,2.085290


In [ ]:
#Intermidiate trials 
        
def SV_pos_int(sv, alpha, beta, A): #This function take as parameters SV, alpha, beta and ambiguity level
    int_pos = {} #probabilities as keys and rewards as a list values.
    for p in p_options: #A loop to iterate through all the probabilities options.
      v = []
      for sv in SV_target: #We want to use differente sv for each prob
        if sv < 0.0:
            v_new = - (abs(sv) / (p - beta * A / 2)) ** (1 / alpha)  # Assuming losses are represented equally than gains!
        else:
            v_new = (sv / (p - beta * A / 2)) ** (1 / alpha)  
        
        if v_new <50.0:
            v.append(v_new)  
        int_pos[p] = v #for every p it includes the calculated value
    return int_pos

int_choicesp = SV_pos_int(sv, alpha,beta,A)

print('Trials with positive subjective values: \n' + str(int_choicesp))

# Convert the dictionary to a DataFrame
dfint = pd.DataFrame(int_choicesp.items(), columns=['p_reward', 'value_reward'])
dfint = dfint.explode('value_reward')
dfint['value_reward'] = dfint['value_reward'].apply(lambda x: np.round(float(x), 2)) #Rounding value_reward
IntTrials = dfint.reset_index(drop=True)
IntTrials['category'] = 'IntTrials'
#Organizing
col = IntTrials.pop('category')
IntTrials.insert(0, col.name,col) 
IntTrials


# 4. The new choices CSV

In [ ]:
crdm_trials = pd.concat([CentTrials, ExtrTrials, IntTrials],ignore_index=True)
crdm_trials['value_safe'] = 5.0
crdm_trials

In [ ]:
# Filter rows where 'value_reward' is less than or equal to 50
crdm_50filtered = crdm_trials[crdm_trials['value_reward'] <= 60]

# Specify the filename for the new CSV file with the filtered data
crdm_trials_50cap = 'csv/crdm_trials_50cap.csv'

# Save the filtered DataFrame to a new CSV file
crdm_50filtered.to_csv(crdm_trials_50cap, index=False)

print(f"CSV file filtered successfully.")
pd.read_csv(crdm_trials_50cap)

# 5. Let's check that it works
# - *a.k.a* Sanity check

In [ ]:
df88 = crdm_trials
df_pos = df88[df88['value_reward']>0].reset_index(drop=True)
df_pos['SV_reward'] = (df_pos['p_reward'] - beta*A/2)*(df_pos['value_reward'])**alpha
df_pos['SV_safe'] = 5.0**alpha
df_pos['deltaSV'] = df_pos['SV_reward'] - df_pos['SV_safe']
df_pos

In [ ]:

df_neg = df88.loc[df88['value_reward']<=0].reset_index(drop=True)
df_neg['value_safe'] = -5.0
df_neg['SV_reward'] = (df_neg['p_reward'] - beta*A/2)*(-1.0)*np.abs(df_neg['value_reward'])**alpha
df_neg['SV_safe'] = -1.0*abs(df_neg['value_safe'])**alpha
df_neg['deltaSV'] = df_neg['SV_reward'] - df_neg['SV_safe']

df_neg['category'] = 'loss'
df_neg

In [ ]:
#Concatenate the 2 trials set
df_posandneg= pd.concat([df_pos,df_neg],ignore_index=True)

#Adding the $50 cap
dfFINAL = df_posandneg[df_posandneg['value_reward'] <= 60]
dfFINAL

In [ ]:
#Scatter Plot
df = df_pos
plt.figure()
x = df['value_reward']
y = df['deltaSV']
plt.scatter(x,y)
plt.xlabel('Value of reward')
plt.ylabel('SV difference (no ambiuguity)')


In [ ]:
from matplotlib.gridspec import GridSpec

def scatter_hist(x, y, ax, ax_histx, ax_histy,xlabel='',ylabel='',title=''):
	ax.scatter(x, y)
	ax.set_xlabel(xlabel,fontsize=14)
	ax.set_ylabel(ylabel,fontsize=14)
	plt.suptitle(title,fontsize=20)
	ax_histx.hist(x,bins=20)
	ax_histy.hist(y, bins=20,orientation = 'horizontal')


def setup_fig_ax():

	fig = plt.figure()
	gs = GridSpec(4, 4)

	ax = fig.add_subplot(gs[1:4, 0:3])
	ax_histx = fig.add_subplot(gs[0,0:3])
	ax_histy = fig.add_subplot(gs[1:4, 3])
	ax_histx.tick_params(axis="x", labelbottom=False)
	ax_histy.tick_params(axis="y", labelleft=False)

	return ax,ax_histx,ax_histy

# delta SV v.s value_reward
x,y = df['value_reward'],df['deltaSV'] 
ax,ax_histx,ax_histy = setup_fig_ax()

scatter_hist(x,y,ax,ax_histx,ax_histy, 
    xlabel='Value of Reward',
    ylabel= 'SV difference',
    title= 'SV difference v.s value of Reward')

In [ ]:
from matplotlib.gridspec import GridSpec
import sys

def scatter_hist(x, y, ax, ax_histx, ax_histy,xlabel='',ylabel='',title=''):
	ax.scatter(x, y)
	ax.set_xlabel(xlabel,fontsize=14)
	ax.set_ylabel(ylabel,fontsize=14)
	plt.suptitle(title,fontsize=20)
	ax_histx.hist(x,bins=20)
	ax_histy.hist(y, bins=20,orientation = 'horizontal')


def setup_fig_ax():

	fig = plt.figure(figsize=(10,8.5))
	gs = GridSpec(4, 4)

	ax = fig.add_subplot(gs[1:4, 0:3])
	ax_histx = fig.add_subplot(gs[0,0:3])
	ax_histy = fig.add_subplot(gs[1:4, 3])
	ax_histx.tick_params(axis="x", labelbottom=False)
	ax_histy.tick_params(axis="y", labelleft=False)

	return ax,ax_histx,ax_histy

# delta SV v.s value_reward
x,y = df['value_reward'],df['deltaSV'] 
ax,ax_histx,ax_histy = setup_fig_ax()

scatter_hist(x,y,ax,ax_histx,ax_histy, 
    xlabel='Value of Reward',
    ylabel= 'SV difference',
    title= 'SV difference v.s value of Reward')

colors = ['r','g','b','c','y','m','k']
df = df.sort_values('value_reward')
print(df['p_reward'].unique())

for i,p in enumerate(reversed(sorted(df['p_reward'].unique()))):
	print('index {} with p_reward is {}'.format(i,p))
	x,y = df.loc[df['p_reward']==p,'value_reward'], df.loc[df['p_reward']==p,'deltaSV']
	ax.plot(x, y,'{}o-'.format(colors[i]),label='prob={0:0.2f}'.format(p))

# ax.plot([50],[],'r-')
ax.axvline(x=50)
# ax.plot([-50,-50],[-10,10],'r-')
ax.axhline(y=0)
# ax.plot([0,50],[0,0],'k--')
ax.legend(loc="lower right")




# To do:
- Is SV symmetrical in the losses --> prospect theory
- What to when SE_trials_reward > $50?
- Add ambiguity to the choice set. 

    

In [ ]:
#Inserting amibuity
import random
df = pd.read_csv(crdm_trials_50cap)

#Define the ambiguity options
crdm_amb_lvl = [24,50,74]


# number of ambiguous trials by type
N_AMB = 10
crdm_amb_opt = [N_AMB*[a] for a in crdm_amb_lvl]
crdm_amb_opt = [item for sublist in crdm_amb_opt for item in sublist]
print(crdm_amb_opt)
random.shuffle(crdm_amb_opt)
print(crdm_amb_opt)



In [ ]:

#Define the number of ambiguous trials by picking X number of random rows.
amb_trials = df.sample(n=len(crdm_amb_opt))

#Assign ambiguity values that match the number of random rows
# amb_levels = [random.choice(crdm_amb_opt) for x in range(15)]
#Assign those values to the random rows
amb_trials['crdm_amb_lev'] = crdm_amb_opt
amb_trials





In [ ]:

crdm_trials['crdm_amb_lev'] = 0.0
# choice_set = crdm_trials
choice_set = pd.concat([crdm_trials,amb_trials],ignore_index=True)

print(choice_set)



In [ ]:
import matplotlib.pyplot as plt

choice_set['SV_lottery'] = (choice_set['p_reward'] - beta*(choice_set['crdm_amb_lev']/100.0/2))*choice_set['value_reward']**alpha
choice_set['SV_safe'] = 5.0**alpha
choice_set['SV_diff'] = choice_set['SV_lottery'] - choice_set['SV_safe']
print(choice_set)

plt.figure(1)
plt.hist(choice_set.loc[choice_set['crdm_amb_lev']==0.0,'SV_diff'],bins=10)

plt.figure(2)
plt.hist(choice_set.loc[choice_set['crdm_amb_lev']>0.0,'SV_diff'],bins=10)



In [ ]:
plt.figure(4)
x = choice_set.loc[choice_set['crdm_amb_lev']==0.0,'value_reward']
y = choice_set.loc[choice_set['crdm_amb_lev']==0.0,'SV_diff']
plt.scatter(x,y)
plt.xlabel('Value of Reward')
plt.ylabel('SV difference (no ambiuguity)')


plt.figure(5)
x = choice_set.loc[choice_set['crdm_amb_lev']>0.0,'crdm_amb_lev']
y = choice_set.loc[choice_set['crdm_amb_lev']>0.0,'SV_diff']
plt.scatter(x,y)
plt.xlabel('Ambiguity level')
plt.ylabel('SV difference (ambiuguity)')



## It's not much but it's honest work
![python](https://plantillasdememes.com/img/plantillas/no-es-mucho-pero-es-trabajo-honesto01569991155.jpg)